In [30]:
import pandas as pd
import numpy as np
import pandas as pd
from datetime import datetime
import warnings

try:

    df_list = []

    for i in range(1, 31):
        try:
            filename = f'resources/transactions_summary ({i}).csv'
            df = pd.read_csv(filename)
            df_list.append(df)
            print(f'Successfully loaded file {filename}')
            # Perform any data analysis or manipulation you need
        except FileNotFoundError:
            pass

    shake_data_df = pd.concat(df_list, ignore_index=True)
    shake_data_df['Date'] = pd.to_datetime(shake_data_df['Date']).dt.strftime('%Y-%m-%d')

except FileNotFoundError:
    print("File not found. Please check the filename and path and try again.")





Successfully loaded file resources/transactions_summary (1).csv
Successfully loaded file resources/transactions_summary (2).csv


In [31]:
# Create a boolean mask for the desired values
mask2 = shake_data_df['Transaction Type'].isin(['shakingstats', 'card cashbacks', 'shakesquads'])

# Apply the mask to the DataFrame to get the filtered rows
filtered_df2 = shake_data_df[mask2]
# Group the filtered dataframe by 'Amount credited' and find the sum, and by 'Spot rate' to find the mean
grouped_df = filtered_df2.groupby('Transaction Type').agg({'Amount Credited': 'sum', 'Spot Rate': 'mean'})

# Print the resulting grouped dataframe
# Create a new column 'Value' that multiplies 'Amount credited' and 'Spot rate'
grouped_df['Value'] = grouped_df['Amount Credited'] * grouped_df['Spot Rate']

# Group the dataframe by 'Transaction Type' where 'Transaction Type' is equal to 'shakingsats'
shakingstats = shake_data_df[shake_data_df['Transaction Type'] == 'shakingsats']


# Group the dataframe by 'Transaction Type' where 'Transaction Type' is equal to 'shakingsats'
shakingstats = shake_data_df[shake_data_df['Transaction Type'] == 'shakingsats'].groupby('Transaction Type').agg({'Amount Credited': 'sum', 'Spot Rate': 'mean'})

# Print the new DataFrame
shakingstats['Value'] = shakingstats['Amount Credited'] * shakingstats['Spot Rate']
merged = pd.concat([grouped_df,shakingstats])
merged

,Amount Credited,Spot Rate,Value
Transaction Type,,,
card cashbacks,0.020218,36287.230367,733.639620
shakesquads,0.000700,32026.444470,22.409544
shakingsats,0.003889,34646.991421,134.724826


In [32]:
shake_data_df1 = shake_data_df.drop(['Debit Currency','Direction', 'Source / Destination', 'Blockchain Transaction ID'], axis=1)
mask = shake_data_df1['Transaction Type'].isin(['purchase/sale'])
new_shake_data_df = shake_data_df[mask]
mask2 = shake_data_df['Credit Currency'].isin(['BTC'])
new_shake_data_df = shake_data_df[mask2]
new_shake_data_df.head(5)
new_shake_data_df.to_csv('resources/new_shake_data_df.csv', index=False)
with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    new_shake_data_df['weightedbuy'] = new_shake_data_df['Buy / Sell Rate'] * new_shake_data_df['Amount Debited']
buy_sell_rate = new_shake_data_df['Buy / Sell Rate'].mean()
stack = new_shake_data_df['Amount Credited'].sum()
cash = new_shake_data_df['Amount Debited'].sum()
weighted_average=new_shake_data_df['weightedbuy'].sum() / new_shake_data_df['Buy / Sell Rate'].mean()

## Group By BTC Total

# Filter the DataFrame based on Transaction Type and Credit Currency
BTC_Total = shake_data_df[(shake_data_df['Transaction Type'] == 'purchase/sale') & (shake_data_df['Credit Currency'] == 'BTC')]
BTC_Total = BTC_Total[BTC_Total['Transaction Type'] == 'purchase/sale'].groupby('Transaction Type').agg({'Amount Debited': 'sum', 'Amount Credited': 'sum', 'Buy / Sell Rate': 'mean'})


## Current Price
import yfinance as yf
btc_df = yf.Ticker('BTC-CAD').history(period='7y',interval='1d',actions=False).reset_index()
btc_df = btc_df.iloc[-1].Close

## Stack

# get input from user and convert to float
my_float_value = float(input("Please enter your float value: "))

# multiply DataFrame by user input and store in Stack variable
Stack = my_float_value * btc_df

## Current Price
import yfinance as yf

btc_df = yf.Ticker('BTC-CAD').history(period='7y',interval='1d',actions=False).reset_index()
Current_Price = btc_df.iloc[-1].Close

## Return
returndf = (Stack/cash)-1

print(f"Current Stack: {(Stack):.4f}")
print(f"Current Price: {(Current_Price):.2f}")
print(f"Current Return: {(returndf * 100):.2f}%")
print(f"Total Invested: {cash:.2f}")
print(f"Weighted Average: {weighted_average:.2f}")
print(f"Average DCA: {buy_sell_rate:.2f}")
print(f"The total value from card cashbacks is {merged.iloc[0,2]:.2f}")
print(f"The total value from card shakesquads is {merged.iloc[1,2]:.2f}")
print(f"The total value from card shakingsats is {merged.iloc[2,2]:.2f}")

new_shake_data_df

Current Stack: 23704.1475
Current Price: 37506.56
Current Return: -11.85%
Total Invested: 26890.94
Weighted Average: 36986.35
Average DCA: 36365.50
The total value from card cashbacks is 733.64
The total value from card shakesquads is 22.41
The total value from card shakingsats is 134.72


,Transaction Type,Date,Amount Debited,Debit Currency,Amount Credited,Credit Currency,Buy / Sell Rate,Direction,Spot Rate,Source / Destination,Blockchain Transaction ID,weightedbuy
9,purchase/sale,2021-05-10,400.0,CAD,0.005553,BTC,72033.0127,purchase,NaN,NaN,NaN,2.881321e+07
12,purchase/sale,2021-05-13,250.0,CAD,0.004047,BTC,61769.6764,purchase,NaN,NaN,NaN,1.544242e+07
15,purchase/sale,2021-05-14,150.0,CAD,0.002441,BTC,61456.2271,purchase,NaN,NaN,NaN,9.218434e+06
18,purchase/sale,2021-05-15,50.0,CAD,0.000827,BTC,60451.1471,purchase,NaN,NaN,NaN,3.022557e+06
20,purchase/sale,2021-05-15,200.0,CAD,0.003322,BTC,60202.2539,purchase,NaN,NaN,NaN,1.204045e+07
...,...,...,...,...,...,...,...,...,...,...,...,...
7127,card cashbacks,2023-10-08,NaN,NaN,0.000004,BTC,NaN,credit,38221.5849,@cashbacks,NaN,NaN
7128,card cashbacks,2023-10-08,NaN,NaN,0.000001,BTC,NaN,credit,38189.1368,@cashbacks,NaN,NaN
7129,card cashbacks,2023-10-08,NaN,NaN,0.000007,BTC,NaN,credit,38120.4370,@cashbacks,NaN,NaN
7130,card cashbacks,2023-10-08,NaN,NaN,0.000004,BTC,NaN,credit,38120.4370,@cashbacks,NaN,NaN


In [33]:




import yfinance as yf

btc_df = yf.Ticker('BTC-CAD').history(period='7y',interval='1d',actions=False).reset_index()
Current_Price = btc_df.iloc[-1].Close
